In [ ]:
%cd PROGRAMS/

In [2]:
import numpy as np
from utils.dataloader import *
from utils.coordinate_calibration import PointCloudRegistration
from utils.meshgrid import Meshgrid
from utils.icp import IterativeClosestPoint, Matching
from utils.deformable_icp import DeformableICP

In [ ]:
OUTPUT_DIR = '../OUTPUT'
DATA_DIR = './pa345_data'

RIGID_BODY_DATA_A = f'{DATA_DIR}/Problem5-BodyA.txt'
RIGID_BODY_DATA_B = f'{DATA_DIR}/Problem5-BodyB.txt'

SURFACE_DATA = f'{DATA_DIR}/Problem5MeshFile.sur'

SAMPLE_ID = 'A'
SAMPLE_DATA = f'./pa345_data/PA5-{SAMPLE_ID}-Debug-SampleReadingsTest.txt'
# SAMPLE_DATA = f'./pa345_data/PA4-{SAMPLE_ID}-Unknown-SampleReadingsTest.txt'

# Load data files
rigidbody_dl_A = RigidBodyDataloader.read_file(RIGID_BODY_DATA_A)
rigidbody_dl_B = RigidBodyDataloader.read_file(RIGID_BODY_DATA_B)

surface_dl = Surfaceloader.read_file(SURFACE_DATA)
sample_dl = SampleReadingsDataloader.read_file(SAMPLE_DATA, delimiter=',', N_A=rigidbody_dl_A.N_markers, N_B=rigidbody_dl_B.N_markers)

modes_dl = AtlasModesDataloader.read_file(f'{DATA_DIR}/Problem5Modes.txt')
print(modes_dl.modes.shape)

### Get F<sub>A, k</sub> and F<sub>B, k</sub>

In [ ]:
rigidbody_dl_A_markers = rigidbody_dl_A.markers.reshape(1, -1, 3) # markers of body A in body A coordinates

sample_dl_A = sample_dl.body_A # samples of body A markers in tracker coordinates
num_samples = sample_dl.N_samps

# perform registration for each frame
reg = PointCloudRegistration(verbose=False)
F_A = []
for i in range(num_samples):
    sample_dl_A_i = sample_dl_A[i].reshape(1, -1, 3)
    F_A_i, err = reg.register(rigidbody_dl_A_markers, sample_dl_A_i)
    F_A.append(F_A_i)

F_A = np.array(F_A)

ridigbody_dl_B_markers = rigidbody_dl_B.markers.reshape(1, -1, 3) # markers of body B in body B coordinates
sample_dl_B = sample_dl.body_B # samples of body B markers in tracker coordinates

# perform registration for each frame
F_B = []
for i in range(num_samples):
    sample_dl_B_i = sample_dl_B[i].reshape(1, -1, 3)

    F_B_i, err = reg.register(ridigbody_dl_B_markers, sample_dl_B_i)
    F_B.append(F_B_i)

F_B = np.array(F_B)

### Get d<sub>k</sub>

In [ ]:
A_tip = rigidbody_dl_A.tip
A_tip = np.append(A_tip, 1) # add 1 for homogenous coordinates
d_k = []

for i in range(num_samples):
    F_A_i = F_A[i] # get F_A for frame i
    F_B_i_inv = np.linalg.inv(F_B[i]) # get F_B inverse for frame i

    d_k.append(F_B_i_inv @ F_A_i @ A_tip) # d_k = F_B^-1 * F_A * A_tip

d_k = np.array(d_k)[:,:3]
print(d_k.shape)

In [ ]:
# Initialize ICP helper class 
deform_icp = DeformableICP()

# Initialize meshgrid of Triangles
mesh = Meshgrid(surface_dl.vertices, surface_dl.triangles)

best_pt_cloud, closest_pt, dist, F_best, λ_best = deform_icp(d_k, mesh, modes_dl.modes)

print(best_pt_cloud)
print(λ_best)

# a, ε, λ_new = deform_icp.test(d_k, mesh, modes_dl.modes)

# print(a)
# print(ε)
# print(λ_new)

# print(Q.shape)
# print(d_k.shape)

# print(d_k[4])
# print(np.sum(Q[4] * λ.reshape(-1, 1), axis=0))

In [ ]:
assert False

### Compute s<sub>k</sub>, c<sub>k</sub>, and d<sub>k</sub>

In [ ]:
matching_algo = Matching.VECTORIZED_LINEAR  # Matching.VECTORIZED_LINEAR is the fastest

# Initialize ICP helper class 
icp = IterativeClosestPoint(match_mode=matching_algo)

# Initialize meshgrid of Triangles
mesh = Meshgrid(surface_dl.vertices, surface_dl.triangles)

best_cloud, closest_pt, dist, F = icp(d_k, mesh)

### write output to file

In [ ]:
output_file = f'{OUTPUT_DIR}/pa3-{SAMPLE_ID}-Output.txt'
with open(output_file, 'w') as file:
    file.write(f"{num_samples}, {output_file}\n")
    for sample in range(num_samples):
        file.write(f"{best_cloud[sample][0]:.2f} {best_cloud[sample][1]:.2f} {best_cloud[sample][2]:.2f} ")
        file.write(f"{closest_pt[sample][0]:.2f} {closest_pt[sample][1]:.2f} {closest_pt[sample][2]:.2f}")
        file.write(f" {dist[sample]:.2f}")
        file.write("\n")

print(f"Output written to {output_file}")
